<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-추출된-속성-전처리" data-toc-modified-id="1.-추출된-속성-전처리-1">1. 추출된 속성 전처리</a></span></li><li><span><a href="#2.-fasttext-토큰화-파일-생성" data-toc-modified-id="2.-fasttext-토큰화-파일-생성-2">2. fasttext 토큰화 파일 생성</a></span></li></ul></div>

In [8]:
import pandas as pd
import pickle
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import itertools
import re
import nltk
from nltk.stem import PorterStemmer
nltk.download('punkt')

In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import tensorflow as tf

# GPU 사용 설정
device_name = tf.test.gpu_device_name()
if not tf.config.list_physical_devices('GPU'):
    print("GPU를 찾을 수 없습니다. 런타임 유형을 GPU로 변경해주세요.")
else:
    print(f"GPU 장치: {device_name}")

GPU 장치: /device:GPU:0


2023-11-17 02:30:58.241064: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-11-17 02:30:58.241095: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-11-17 02:30:58.241102: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-11-17 02:30:58.241186: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-17 02:30:58.241384: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# 1. 추출된 속성 전처리
PyABSA에서 ATEPC모델을 사용해서 추출한 속성에 대해 전처리 수행  

- **처리 프로세스**
    1. 소문자 변환 (ex: Battery Capacity!! -> battery capacity!!)
    2. 특수 기호 제거 (ex: battery capacity -> battery capacity)
    3. 스테밍 (ex: battery capacity -> batteri capac)
    4. 띄어쓰기 언더바('_') 대체 (ex: batteri_capac)
    4. 일부 전치사(불용어) 제거 (ex: size of the screen -> size_screen)
    
<br>

- **추출된 속성 정리 (열 설명)**  
    - aspect : 실제 문장 내 속성 형태
    - ori_aspect : 스테밍 없이 띄어쓰기 '_'로 대체
    - pre_aspect : 스테밍, 전치사 제거까지 수행 (속성 사전에서 분류 속성으로 사용)
    - count : aspect가 등장한 리뷰 수
    - all_cnt : 전처리된 속성이 등장한 리뷰 수 


In [3]:
data = pd.read_csv('/Users/sonjiyeong/Documents/랩실 프로젝트/스마트폰/code/감성분석후_데이터프레임.csv')

# 1. 소문자 변환
data['tokens'] = data['tokens'].str.lower()
data['aspect'] = data['aspect'].str.lower()

import ast
# 리스트 형태로 변환 (데이터 저장 상태에 따라 선택적으로 필요)
data['tokens'] = data['tokens'].apply(lambda x: ast.literal_eval(x)) 
data['aspect'] = data['aspect'].apply(lambda x: ast.literal_eval(x))

In [4]:
df = data.copy()

In [5]:
df.head(3)

,id,model,date,rating,sentence,IOB,tokens,aspect,position,sentiment,probs,confidence
0,1,galaxy_s3,2012-05-11,2,too plasticky for $ 700 . When I put a review ...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[too, plasticky, for, $, 700, ., when, i, put,...","[costs, materials]","[[45], [52]]","['Negative', 'Negative']","[[0.9986786246299744, 0.000984302954748273, 0....","[0.9987, 0.9983]"
1,2,galaxy_s3,2012-06-01,5,So far super good . I got my S3 in white yeste...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[so, far, super, good, ., i, got, my, s3, in, ...","[memory, battery, battery life]","[[48], [57], [59, 60]]","['Neutral', 'Positive', 'Positive']","[[0.0006813739310018718, 0.5115107893943787, 0...","[0.5115, 0.9756, 0.9979]"
2,3,galaxy_s3,2012-06-01,2,Expensive or Respect . i ' ve narrowed my choi...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[expensive, or, respect, ., i, ', ve, narrowed...","[iphone, customer service]","[[31], [59, 60]]","['Neutral', 'Negative']","[[0.006004048511385918, 0.9936121106147766, 0....","[0.9936, 0.9988]"


In [6]:
def replace_word_in_list(word_list, target_word):
    for i, word in enumerate(word_list):
        if word == target_word:
            word_list[i] = 'NaN'

# 오류나는 일부 데이터 변경하기 (선택사항)
target_word = '\n \n'  
for index, row in df.iterrows():
    replace_word_in_list(row['aspect'], target_word)

target_word = '\n'  
for index, row in df.iterrows():
    replace_word_in_list(row['aspect'], target_word)

target_word = '\xa0'  
for index, row in df.iterrows():
    replace_word_in_list(row['aspect'], target_word)

In [7]:
df_savepoint = df.copy()

In [8]:
df = df_savepoint.copy()

In [9]:
# 2. 특수기호 제거 함수
def ori_remove_special_chars(list_aspect):
    new_list = []
    for aspect in list_aspect:
        
        aspect = aspect.split()
        aspect = '_'.join(aspect)
        
        aspect = re.sub(r'[^a-zA-Z0-9_]', '', aspect)
        aspect = aspect.replace('___', '_')
        aspect = aspect.replace('__', '_')
        aspect = aspect.strip('_') ####
        new_list.append(aspect)
    return new_list


# 3. 스테밍 함수
def stem_words(word_list):
    stemmer = PorterStemmer()
    stemmed_words = []
    for word in word_list:
        if ' ' in word:
            compound_word = ""
            for w in word.split(' '):
                compound_word = compound_word + ' ' + stemmer.stem(w)
            compound_word = compound_word.strip()
            stemmed_words.append(compound_word)
        else:
            stemmed_words.append(stemmer.stem(word))
    
    return stemmed_words

# 4. 띄어쓰기 언더바 대체 
def replace_spaces_with_underscore(lst):
    return [word.replace(' ', '_') for word in lst]

def pre_remove_special_chars(list_aspect):
    new_list = []
    for aspect in list_aspect:
        aspect = re.sub(r'[^a-zA-Z0-9_]', '', aspect)
        aspect = '_' + aspect + '_'
        aspect = aspect.replace('_of_', '_').replace('_in_', '_').replace('_the_', '_')
        aspect = aspect.replace('_for_', '_').replace('_and_', '_').replace('_at_', '_')
        aspect = aspect.replace('___', '_')
        aspect = aspect.replace('__', '_')
        aspect = aspect.strip('_')
        new_list.append(aspect)
    return new_list
# 삭제한 전치사 : of, in, the, at, for, and

In [10]:
import itertools

with tf.device(device_name):
    all_as_lst = list(itertools.chain.from_iterable(df['aspect']))
    ori_as_lst = ori_remove_special_chars(all_as_lst) # 2. 특수기호 제거
    as_lst = stem_words(all_as_lst) # 3. 스테밍
    pre_as_lst = replace_spaces_with_underscore(as_lst) # 4. 띄어쓰기 언더바('_') 대체
    pre_as_lst = pre_remove_special_chars(pre_as_lst) # 5. (특수기호 제거) & 일부 전치사(불용어)

In [11]:
print(len(ori_as_lst))
print(len(as_lst))
print(len(pre_as_lst))

132956
132956
132956


In [12]:
df_ori_dic = pd.DataFrame({'aspect':all_as_lst, 'ori_aspect':ori_as_lst, 'pre_aspect':pre_as_lst})
df_ori_dic.replace('', np.nan, inplace=True)
df_ori_dic.dropna(subset=['ori_aspect'], inplace=True)

In [13]:
df_dic_cnt = df_ori_dic.groupby(['aspect', 'ori_aspect', 
            'pre_aspect'])['ori_aspect'].size().reset_index(name='count').sort_values(by='count', ascending=False)
df_all_cnt = df_ori_dic['pre_aspect'].value_counts().reset_index().rename(columns={'count':'all_cnt'})
df_dic_cnt = pd.merge(df_dic_cnt, df_all_cnt, on='pre_aspect').sort_values(by='count', ascending=False)
df_dic_cnt = df_dic_cnt.reset_index(drop=True)
df_dic_cnt

,aspect,ori_aspect,pre_aspect,count,all_cnt
0,phone,phone,phone,10667,10850
1,battery,battery,batteri,9048,9255
2,screen,screen,screen,5880,5956
3,works,works,work,5843,8067
4,iphone,iphone,iphon,4835,5024
...,...,...,...,...,...
9356,snap in protection,snap_in_protection,snap_protect,1,1
9357,system android,system_android,system_android,1,1
9358,syste,syste,syst,1,1
9359,snapdragon 400,snapdragon_400,snapdragon_400,1,1


In [14]:
df_dic_cnt.to_csv('추출속성_데이터프레임.csv', index=False)

# 2. fasttext 토큰화 파일 생성
기존 토큰화된 문장에서 띄어쓰기가 존재하는 합성명사를 '_'로 대체한 토큰을 사용하여 단어 유사도를 도출
- 처리 프로세스
    - '_'를 포함한 토큰으로 변경  
        (ex: battery capacity -> battery_capacity)


- 처리 이유
    - fasttext 유사 단어를 찾을 때 합성명사를 잘 고려하지 못함  
        => ATEPC 모델로 추출된 합성어의 띄어쓰기는 '_'로 대체하여 fasttext 모델이 한 단어로 인식하도록 구현   
        => 이렇게 처리할 경우 합성어와 일반명사를 복합적으로 고려하여 유사한 단어를 추출해준다는 것을 확인함


In [16]:
def ori_create_compound_words(word_list, compound_list):
    combined_words = list(word_list)
    for compound in compound_list:
        words = compound.split()
        compound_word = '_'.join(words)  # 띄어쓰기를 언더스코어로 대체
        compound_word = re.sub(r'[^a-zA-Z0-9_]', '', compound_word)
        compound_word = compound_word.replace('___', '_')
        compound_word = compound_word.replace('__', '_')
        compound_word = compound_word.strip('_') ####
        
        idx = 0
        while idx < len(combined_words):
            if combined_words[idx:idx+len(words)] == words:
                combined_words[idx:idx+len(words)] = [compound_word]
                idx += len(words)
            else:
                idx += 1
    return combined_words

In [17]:
with tf.device(device_name):
    # 데이터프레임의 두 열에 함수 적용
    df['new_tokens'] = df.apply(lambda row: ori_create_compound_words(row['tokens'], row['aspect']), axis=1)

In [11]:
df.head(3)
df.shape

,id,model,date,rating,sentence,IOB,tokens,aspect,position,sentiment,probs,confidence,new_tokens
0,1,galaxy_s3,2012-05-11,2,too plasticky for $ 700 . When I put a review ...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[too, plasticky, for, $, 700, ., when, i, put,...","[costs, materials]","[[45], [52]]","[Negative, Negative]","[[0.9986786246299744, 0.000984302954748273, 0....","[0.9987, 0.9983]","[too, plasticky, for, $, 700, ., when, i, put,..."
1,2,galaxy_s3,2012-06-01,5,So far super good . I got my S3 in white yeste...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[so, far, super, good, ., i, got, my, s3, in, ...","[memory, battery, battery life]","[[48], [57], [59, 60]]","[Neutral, Positive, Positive]","[[0.0006813739310018718, 0.5115107893943787, 0...","[0.5115, 0.9756, 0.9979]","[so, far, super, good, ., i, got, my, s3, in, ..."
2,3,galaxy_s3,2012-06-01,2,Expensive or Respect . i ' ve narrowed my choi...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[expensive, or, respect, ., i, ', ve, narrowed...","[iphone, customer service]","[[31], [59, 60]]","[Neutral, Negative]","[[0.006004048511385918, 0.9936121106147766, 0....","[0.9936, 0.9988]","[expensive, or, respect, ., i, ', ve, narrowed..."


(76215, 13)

In [18]:
df['new_tokens'][1] # battery life -> battery_life

['so',
 'far',
 'super',
 'good',
 '.',
 'i',
 'got',
 'my',
 's3',
 'in',
 'white',
 'yesterday',
 'i',
 'had',
 'the',
 'iphone',
 '4s',
 'before',
 'but',
 'i',
 'played',
 'before',
 'with',
 'the',
 's2',
 'that',
 'i',
 'already',
 'liked',
 '.',
 'the',
 'phone',
 'feels',
 'great',
 'in',
 'hand',
 'it',
 "'",
 's',
 'super',
 'fast',
 '.',
 'i',
 'like',
 'the',
 'fact',
 'to',
 'expand',
 'memory',
 'by',
 'micro',
 'sd',
 'and',
 'that',
 'you',
 'can',
 'change',
 'battery',
 '.',
 'battery_life',
 'is',
 'great',
 'for',
 'me',
 'so',
 'far',
 'looking',
 'forward',
 'to',
 'explore',
 'more',
 'features',
 '.',
 'in',
 'case',
 'you',
 'have',
 'the',
 'htc',
 'one',
 'x',
 'i',
 'think',
 'you',
 'don',
 "'",
 't',
 'need',
 'to',
 'upgrade',
 'otherwise',
 'i',
 'can',
 'highly',
 'recommend',
 'it',
 'it',
 "'",
 's',
 'not',
 'feeling',
 'so',
 'huge',
 'than',
 'the',
 'note',
 '.']

In [ ]:
import pickle
file_name = '감성분석후_속성전처리_데이터프레임.pkl'

# 데이터를 pickle 파일로 저장
with open(file_name, 'wb') as file:
    pickle.dump(df, file)

df.to_csv('감성분석후_속성전처리_데이터프레임.csv', index=False)

In [13]:
with open('tokenized_data.txt', 'w') as f:
    for sentence_tokens in df['new_tokens']:
        sentence = ' '.join(sentence_tokens)
        f.write(sentence + '\n')

- 유사한 단어를 찾아주는 model 생성 
- tokenized_data.txt 파일로 모델 학습

In [17]:
import fasttext
with tf.device(device_name) :
    model = fasttext.train_unsupervised('tokenized_data.txt', epoch=10, dim=300, lr=0.1, wordNgrams=1, minCount=5)

Read 4M words
Number of words:  10730
Number of labels: 0
Progress: 100.0% words/sec/thread:   39994 lr:  0.000000 avg.loss:  2.298994 ETA:   0h 0m 0s


In [62]:
model.get_nearest_neighbors('standy_time', k=50)

[(0.6286474466323853, 'stands'),
 (0.621839165687561, 'stand'),
 (0.5788232684135437, 'standing'),
 (0.5265442132949829, 'standby'),
 (0.45848798751831055, 'standpoint'),
 (0.4474009573459625, 'overtime'),
 (0.42048245668411255, 'standard'),
 (0.41055670380592346, 'behind'),
 (0.40568146109580994, 'lifetime'),
 (0.40383702516555786, 'longtime'),
 (0.4015600383281708, 'meantime'),
 (0.382270872592926, 'time'),
 (0.38153693079948425, 'standards'),
 (0.3811284005641937, 'ontime'),
 (0.374737411737442, 'sgiii'),
 (0.3687078654766083, 'prolong'),
 (0.3679945468902588, 'caring'),
 (0.36645665764808655, 'candy'),
 (0.36356571316719055, 'leave'),
 (0.3583761155605316, 'blink'),
 (0.35614821314811707, 'crowd'),
 (0.3546993136405945, 'facetime'),
 (0.35464122891426086, 'challenges'),
 (0.35408419370651245, 'that'),
 (0.35036492347717285, 'bettery'),
 (0.35003411769866943, 'surpasses'),
 (0.34884828329086304, 'stats'),
 (0.34786149859428406, 'competent'),
 (0.347148060798645, 'understanding'),
 (

In [66]:
model2.get_nearest_neighbors('logic_board', k=50)

[(0.6590221524238586, 'main_board'),
 (0.6465955376625061, 'mother_board'),
 (0.6040922999382019, 'circuit_board'),
 (0.5638992786407471, 'mother_boards'),
 (0.5462538599967957, 'motherboard'),
 (0.5304113030433655, 'logic'),
 (0.4723176658153534, 'board'),
 (0.45542261004447937, 'diagnosis'),
 (0.42834359407424927, 'unfixable'),
 (0.42079854011535645, 'recalled'),
 (0.41768550872802734, 'diagnosed'),
 (0.41749799251556396, 'recall'),
 (0.40325868129730225, 'onboard'),
 (0.39644157886505127, 'service_center'),
 (0.3960813581943512, 'faulty'),
 (0.39529648423194885, 'irreparable'),
 (0.38819706439971924, 'diagnose'),
 (0.3867478668689728, 'repair'),
 (0.38547632098197937, 'serviced'),
 (0.38326549530029297, 'repairable'),
 (0.37284427881240845, 'flipboard'),
 (0.36539363861083984, 'internal_parts'),
 (0.3653269410133362, 'lithium'),
 (0.36451321840286255, '319'),
 (0.3615002930164337, 'corrosion'),
 (0.3598882853984833, '66'),
 (0.3590587079524994, 'technician'),
 (0.35351651906967163, 